**LCEL 이란 대화 내용을 기억하는 메모리를 추가하는 것을 말합니다.**



In [6]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화합니다.

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)
model = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-3.5-turbo",  # 모델명
)

chain = runnable | prompt | model

output = chain.invoke({"input" : "만나서 반갑습니다. 제 이름은 현상입니다."})


In [8]:
output

AIMessage(content='만나서 반가워요, 현상님! 무엇을 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 33, 'total_tokens': 63}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7de48c99-3e8c-4b25-8eb7-09319bdbc379-0', usage_metadata={'input_tokens': 33, 'output_tokens': 30, 'total_tokens': 63})

In [7]:
memory.load_memory_variables({})

{'chat_history': []}

In [9]:
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 현상입니다."}, {"ai": output.content}
)

In [10]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 현상입니다.'),
  AIMessage(content='만나서 반가워요, 현상님! 무엇을 도와드릴까요?')]}

In [11]:
print(chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"}))

content='네, 현상님의 이름을 기억하고 있어요! 어떻게 도와드릴까요?' response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 88, 'total_tokens': 124}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3b83c0c9-a016-4d7f-89c8-47dfea2a0a2e-0' usage_metadata={'input_tokens': 88, 'output_tokens': 36, 'total_tokens': 124}


**LCEL 가지고 나만의 Chain 만들기**

In [12]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
class myConversationChain(Runnable):
    def __init__(self, model, prompt, memory):
        self.prompt = prompt
        self.memory = memory
        self.model = model 
        self.runnable = RunnablePassthrough.assign(
            chat_history=RunnableLambda(memory.load_memory_variables)
            | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
        )
        self.chain = runnable | prompt | model | StrOutputParser()

    
    def invoke(self, query, ):
        answer = self.chain.invoke({"input": query})
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer



    

In [16]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화합니다.

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)
model = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
)

chain = myConversationChain(model , prompt , memory)

output = chain.invoke("제이름은 현상입니다.")

In [17]:
print(output)

print(chain.invoke("제 이름이 뭐였죠?"))

안녕하세요, 현상님! 만나서 반갑습니다. 어떻게 도와드릴까요?
당신의 이름은 현상입니다. 어떻게 도와드릴까요?
